In [1]:
import pandas as pd
from scipy.optimize import curve_fit
from scipy.integrate import quad
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
from datetime import datetime, date

C:\Users\etiirine\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Reading in the data

In [2]:
#read in the data
mid = pd.read_csv("MID.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
mid.drop([0,1], inplace=True) #to get rid of the spikes in the beginning

#bring the time data into a new df
time = pd.DataFrame(mid.iloc[:,0::2]) #- get every second column

#add elapsed time columns
def elapsed_time(df):
    itr = 0
    counter = len(df.columns)
    offset = [475.157,475.157,475.157,475.157,475.157,475.158,475.658]#max value of "Elapsed time" in baseline MID
    masses = [2,12,15,18,28,32,44]
    for i in df:
        timestamp = pd.to_datetime(df[i], format="%Y/%m/%d %H:%M:%S.%f")
        print(timestamp)
        runtime = offset[itr]+(timestamp-timestamp[2]).dt.total_seconds()
        colname = "Elapsed time, mass {}".format(masses[itr]) 
        df.insert(counter, colname, runtime)
        itr += 1
        counter += 1
    return(df)
elapsed_time(time)
#time["Elapsed time, mass 40"].describe()

2       2023-02-24 17:20:38.457
3       2023-02-24 17:20:43.430
4       2023-02-24 17:20:48.404
5       2023-02-24 17:20:53.378
6       2023-02-24 17:20:58.352
                  ...          
10566   2023-02-25 07:56:21.407
10567   2023-02-25 07:56:26.381
10568   2023-02-25 07:56:31.355
10569   2023-02-25 07:56:36.329
10570   2023-02-25 07:56:41.303
Name: mass 2.0_time, Length: 10569, dtype: datetime64[ns]
2       2023-02-24 17:20:41.658
3       2023-02-24 17:20:46.631
4       2023-02-24 17:20:51.605
5       2023-02-24 17:20:56.579
6       2023-02-24 17:21:01.553
                  ...          
10566   2023-02-25 07:56:24.608
10567   2023-02-25 07:56:29.582
10568   2023-02-25 07:56:34.556
10569   2023-02-25 07:56:39.530
10570   2023-02-25 07:56:44.504
Name: mass 12.0_time, Length: 10569, dtype: datetime64[ns]
2       2023-02-24 17:20:41.858
3       2023-02-24 17:20:46.831
4       2023-02-24 17:20:51.805
5       2023-02-24 17:20:56.779
6       2023-02-24 17:21:01.753
                  .

,mass 2.0_time,mass 12.0_time,mass 15.0_time,mass 18.0_time,mass 28.0_time,mass 32.0_time,mass 44.0_time,"Elapsed time, mass 2","Elapsed time, mass 12","Elapsed time, mass 15","Elapsed time, mass 18","Elapsed time, mass 28","Elapsed time, mass 32","Elapsed time, mass 44"
2,2023/02/24 17:20:38.457,2023/02/24 17:20:41.658,2023/02/24 17:20:41.858,2023/02/24 17:20:42.058,2023/02/24 17:20:42.258,2023/02/24 17:20:42.458,2023/02/24 17:20:43.159,475.157,475.157,475.157,475.157,475.157,475.158,475.658
3,2023/02/24 17:20:43.430,2023/02/24 17:20:46.631,2023/02/24 17:20:46.831,2023/02/24 17:20:47.031,2023/02/24 17:20:47.232,2023/02/24 17:20:47.432,2023/02/24 17:20:48.133,480.130,480.130,480.130,480.130,480.131,480.132,480.632
4,2023/02/24 17:20:48.404,2023/02/24 17:20:51.605,2023/02/24 17:20:51.805,2023/02/24 17:20:52.005,2023/02/24 17:20:52.205,2023/02/24 17:20:52.405,2023/02/24 17:20:53.107,485.104,485.104,485.104,485.104,485.104,485.105,485.606
5,2023/02/24 17:20:53.378,2023/02/24 17:20:56.579,2023/02/24 17:20:56.779,2023/02/24 17:20:56.980,2023/02/24 17:20:57.180,2023/02/24 17:20:57.380,2023/02/24 17:20:58.081,490.078,490.078,490.078,490.079,490.079,490.080,490.580
6,2023/02/24 17:20:58.352,2023/02/24 17:21:01.553,2023/02/24 17:21:01.753,2023/02/24 17:21:01.953,2023/02/24 17:21:02.153,2023/02/24 17:21:02.353,2023/02/24 17:21:03.055,495.052,495.052,495.052,495.052,495.052,495.053,495.554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10566,2023/02/25 07:56:21.407,2023/02/25 07:56:24.608,2023/02/25 07:56:24.808,2023/02/25 07:56:25.008,2023/02/25 07:56:25.208,2023/02/25 07:56:25.408,2023/02/25 07:56:26.110,53018.107,53018.107,53018.107,53018.107,53018.107,53018.108,53018.609
10567,2023/02/25 07:56:26.381,2023/02/25 07:56:29.582,2023/02/25 07:56:29.782,2023/02/25 07:56:29.982,2023/02/25 07:56:30.182,2023/02/25 07:56:30.382,2023/02/25 07:56:31.084,53023.081,53023.081,53023.081,53023.081,53023.081,53023.082,53023.583
10568,2023/02/25 07:56:31.355,2023/02/25 07:56:34.556,2023/02/25 07:56:34.756,2023/02/25 07:56:34.956,2023/02/25 07:56:35.157,2023/02/25 07:56:35.357,2023/02/25 07:56:36.058,53028.055,53028.055,53028.055,53028.055,53028.056,53028.057,53028.557
10569,2023/02/25 07:56:36.329,2023/02/25 07:56:39.530,2023/02/25 07:56:39.730,2023/02/25 07:56:39.930,2023/02/25 07:56:40.130,2023/02/25 07:56:40.330,2023/02/25 07:56:41.032,53033.029,53033.029,53033.029,53033.029,53033.029,53033.030,53033.531


In [3]:
# #read in the baseline MID
# basel = pd.read_csv("baseline MID.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
# basel.drop([0,1], inplace=True)

# #bring the time data into a new df
# time1 = pd.DataFrame(basel.iloc[:,0::2]) #- get every second column

# #add elapsed time columns
# def elapsed_time(df):
#     itr = 0
#     counter = len(df.columns)
#     masses = [2,12,15,18,28,32,44]
#     for i in df:
#         timestamp = pd.to_datetime(df[i], format="%Y/%m/%d %H:%M:%S.%f")
#         runtime = (timestamp-timestamp[2]).dt.total_seconds()
#         colname = "Elapsed time, mass {}".format(masses[itr]) 
#         df.insert(counter, colname, runtime)
#         itr += 1
#         counter += 1
#     return(df)
# elapsed_time(time1)

# #time["Elapsed time, mass 40"].describe()

In [4]:
#reading the Emission current, pressures
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()

#adding an elapsed time column
timestamp=[]
for i in vaclog["Time"]:
    timestamp.append(i)
#making the column into a datetime object
timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
vaclog.drop(index=vaclog.index[0], 
        axis=0, 
        inplace=True)
#creating a new df by dropping the rows where system was warm
cut_vaclog = vaclog.loc[vaclog["Elapsed time"] >= 3310]
cut_vaclog.head()

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
479,NaN,24/02/2023 18:15:14,3311.0,78.313,1.640000e-08,7.030000e-09,7.578000e-09,3.652000e-07,1.1,361.0,-5.163,4308.176,NaN,NaN
480,NaN,24/02/2023 18:15:21,3318.0,78.310,1.640000e-08,7.050000e-09,7.569000e-09,3.652000e-07,1.1,361.0,-5.163,4308.176,NaN,NaN
481,NaN,24/02/2023 18:15:28,3325.0,78.309,1.630000e-08,7.070000e-09,7.541000e-09,3.652000e-07,1.1,361.0,-5.275,4308.288,NaN,NaN
482,NaN,24/02/2023 18:15:35,3332.0,78.310,1.640000e-08,7.050000e-09,7.569000e-09,3.661000e-07,1.1,361.0,-5.275,4308.176,NaN,NaN
483,NaN,24/02/2023 18:15:42,3339.0,78.309,1.640000e-08,7.030000e-09,7.541000e-09,3.652000e-07,1.1,361.0,-5.050,4308.176,NaN,NaN


### Data processing

In [5]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


In [6]:
# #Combining the time df's
# t_comb = pd.concat([time1,time],join="inner")
# #t_comb.drop([0,1,91,92], inplace=True)

# t_comb

In [7]:
# #Combining the MIDs
# #read in the baseline MID
# comb = pd.read_csv("combined MID.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
# comb.drop([0,1,91,92], inplace=True)
# print(comb)


### Plotting

In [8]:
#Plotting the all the relevant MID spectrums

plt.figure(figsize=(14,8))
plt.plot(time['Elapsed time, mass 2'],mid["mass 2.0_value"],marker=".", markersize=4,label='M2')
plt.plot(time['Elapsed time, mass 12'],mid["mass 12.0_value"],marker=".", markersize=4,label='M12')
plt.plot(time['Elapsed time, mass 15'],mid["mass 15.0_value"],marker=".", markersize=4,label='M15')
plt.plot(time['Elapsed time, mass 18'],mid["mass 18.0_value"],marker=".", markersize=4,label='M18')
plt.plot(time['Elapsed time, mass 28'],mid["mass 28.0_value"],marker=".", markersize=4,label='M28')
plt.plot(time['Elapsed time, mass 32'],mid["mass 32.0_value"],marker=".", markersize=4,label='M32')
plt.plot(time["Elapsed time, mass 44"],mid["mass 44.0_value"],marker=".", markersize=4,label='M44')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Ion current (A)')
plt.legend(title="Mass fractions", loc="upper right")
plt.title("TDS MID scan no pre-injection, dynamic vacuum")

plt.yscale('log')

In [9]:
# #Plotting the baseline mid

# plt.figure(figsize=(14,8))
# plt.plot(time1['Elapsed time, mass 2'],basel["mass 2.0_value"],marker=".", markersize=4,label='M2')
# plt.plot(time1['Elapsed time, mass 12'],basel["mass 12.0_value"],marker=".", markersize=4,label='M12')
# plt.plot(time1['Elapsed time, mass 15'],basel["mass 15.0_value"],marker=".", markersize=4,label='M15')
# plt.plot(time1['Elapsed time, mass 18'],basel["mass 18.0_value"],marker=".", markersize=4,label='M18')
# plt.plot(time1['Elapsed time, mass 28'],basel["mass 28.0_value"],marker=".", markersize=4,label='M28')
# plt.plot(time1['Elapsed time, mass 32'],basel["mass 32.0_value"],marker=".", markersize=4,label='M32')
# plt.plot(time1["Elapsed time, mass 44"],basel["mass 44.0_value"],marker=".", markersize=4,label='M44')

# plt.xlabel('Elapsed time (s)')
# plt.ylabel('Ion current (A)')
# plt.legend(title="Mol masses", loc="upper right")
# plt.title("TDS MID baseline scan no pre-injection, dynamic vacuum")

# plt.yscale('log')

In [10]:
# #Plotting the combined MID
# plt.figure(figsize=(14,8))
# plt.plot(t_comb['Elapsed time, mass 2'],comb["mass 2.0_value"],marker=".", markersize=4,label='M2')
# plt.plot(t_comb['Elapsed time, mass 12'],comb["mass 12.0_value"],marker=".", markersize=4,label='M12')
# plt.plot(t_comb['Elapsed time, mass 15'],comb["mass 15.0_value"],marker=".", markersize=4,label='M15')
# plt.plot(t_comb['Elapsed time, mass 18'],comb["mass 18.0_value"],marker=".", markersize=4,label='M18')
# plt.plot(t_comb['Elapsed time, mass 28'],comb["mass 28.0_value"],marker=".", markersize=4,label='M28')
# plt.plot(t_comb['Elapsed time, mass 32'],comb["mass 32.0_value"],marker=".", markersize=4,label='M32')
# plt.plot(t_comb["Elapsed time, mass 44"],comb["mass 44.0_value"],marker=".", markersize=4,label='M44')

# plt.xlabel('Elapsed time (s)')
# plt.ylabel('Ion current (A)')
# plt.legend(title="Mol masses", loc="upper right")
# plt.title("TDS Combined baseline and MID scans no pre-injection, dynamic vacuum")

# plt.yscale('log')

In [11]:
#Plotting BA2, BA1 pressure exc time when warm
plt.figure(figsize=(10,6))
plt.plot(cut_vaclog["Elapsed time"],cut_vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(cut_vaclog["Elapsed time"],cut_vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("TDS Total pressure evolution 2e18 M pre-injection, EGA fully in, dynamic vacuum")
ax = plt.gca()
ax.set_yscale('log')
plt.tick_params(axis="y", which='minor')
plt.grid(visible=None, which='both', axis='both')
plt.show()

In [16]:
# Plotting the temp evolution exc warm part
plt.figure()
plt.plot((cut_vaclog["Elapsed time"]),polyfit(A,cut_vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("TDS Temperature evolution 2e18 M pre-injection, EGA fully in, dynamic vacuum")
plt.xlim(0,400000)
plt.ylim(0,300)
plt.yscale('linear')